In [38]:
import os
os.chdir("/home/team_thuctap/ltlinh/new_ndanh")  # nếu notebook nằm trong src/error_analysis

In [39]:
import os
print(os.getcwd())

/home/team_thuctap/ltlinh/new_ndanh


In [40]:
import sys
from pprint import pprint
pprint(sys.path)

['/home/team_thuctap/miniconda3/envs/linh_env/lib/python310.zip',
 '/home/team_thuctap/miniconda3/envs/linh_env/lib/python3.10',
 '/home/team_thuctap/miniconda3/envs/linh_env/lib/python3.10/lib-dynload',
 '',
 '/home/team_thuctap/miniconda3/envs/linh_env/lib/python3.10/site-packages']


In [44]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import argparse
from src.core.data_manager import DataProcessor

def plot_interactive_area_ratio(
    df: pd.DataFrame,
    out_html: Path,
    bins: int = 10000
):
    """
    df columns: [area_ratio, class]
    """

    # giữ toàn bộ dữ liệu hợp lệ
    df = df[(df["area_ratio"] > 0) & (df["area_ratio"] <= 1)]

    bin_size = 1.0 / bins

    fig = px.histogram(
        df,
        x="area_ratio",
        color="class",
        opacity=0.7,
        barmode="stack",     # nếu muốn so tổng
        histnorm=None
    )

    fig.update_traces(
        xbins=dict(start=0.0, end=1.0, size=1.0 / bins)
    )

    fig.update_layout(
        title="Bounding Box Area Ratio Distribution (Zoomable)",
        xaxis_title="bbox_area / image_area",
        yaxis_title="Number of bounding boxes",
        bargap=0,
        template="plotly_white",
    )

    # cho phép zoom mượt
    fig.update_xaxes(
        rangeslider_visible=True   # 🔥 thanh zoom dưới
    )

    out_html.parent.mkdir(parents=True, exist_ok=True)
    fig.write_html(out_html)

    print(f"[DONE] Interactive plot saved to {out_html}")


def main():

    data = "/home/team_thuctap/ltlinh/new_ndanh/data/merged_data/v7_filterBB_normalized/data.yaml"
    out_dir = "/home/team_thuctap/ltlinh/new_ndanh/data/merged_data/v7_filterBB_normalized/gt_area_analysis"
    # ---- Load GT bbox data ----
    dp = DataProcessor(data)
    dims = dp.compute_bbox_dimensions(["val"])
    print(len(dims))
    # (w_px, h_px, area, img_area, class_name)

    if len(dims) == 0:
        print("[ERROR] No bounding boxes found.")
        return

    df = pd.DataFrame(
        dims,
        columns=["width", "height", "area", "img_area", "class"]
    )
    df["area_ratio"] = df["area"] / df["img_area"]

    out_html = Path(out_dir) / "val_area_ratio_zoomable.html"

    plot_interactive_area_ratio(
        df[["area_ratio", "class"]],
        out_html,
        bins=10000
    )
if __name__ == "__main__":
    main()


27128
[DONE] Interactive plot saved to /home/team_thuctap/ltlinh/new_ndanh/data/merged_data/v7_filterBB_normalized/gt_area_analysis/val_area_ratio_zoomable.html


In [ ]:
103137


130265